<img src="./images/DLI_Header.png" style="width: 400px;">

## 评估
恭喜您完成了今天的课程！希望您在此过程中学到了一些有价值的技能。现在该测试一下这些技能了。在此评估中，您将训练一种能够识别新鲜和腐烂水果的新模型。您需要使模型的验证准确率达到92％，才能通过评估，但我们鼓励您挑战更高的准确率。为此，您将使用先前练习中学到的技能，具体来说，我们建议您结合使用迁移学习、数据扩充和模型微调。训练好模型并在测试数据集上的准确率达到至少92％之后，请保存模型，然后评估其准确率。让我们开始吧！

### 数据集
在本练习中，您将训练一个模型来识别新鲜和腐烂的水果，数据集来自[Kaggle](https://www.kaggle.com/sriramr/fruits-fresh-and-rotten-for-classification)。如果您有兴趣在课后自己开始一个新的项目，那么Kaggle是一个值得访问的好地方。现在您可详细查看`data/fruits`文件夹中的数据集结构。水果有六类：新鲜的苹果，新鲜的橙子，新鲜的香蕉，烂的苹果，烂的橙子和烂的香蕉。这意味着您的模型将需要有6个神经元的输出层才能成功进行分类 您还需要使用`categorical_crossentropy`作为损失函数来编译模型，因为我们有两个以上的类别。

<img src="./images/fruits.png" style="width: 600px;">

### 加载ImageNet预训练的基础模型
我们鼓励您从在ImageNet上预训练的模型开始。您需要用正确的权重加载模型，设置输入的形状，然后选择删除模型的最后一层。请记住，图像具有三个维度：高度和宽度以及多个颜色通道。因为这些图片是彩色的，所以会有红色，绿色和蓝色三个通道。我们已经为您填写了输入形状，请不要更改，否则评估将失败。如果您需要预训练模型的参考设置，请查看笔记本05b，您在那里最先实现的迁移学习。

In [16]:
# from tensorflow import keras

# base_model = keras.applications.VGG16(
#     weights=FIXME,
#     input_shape=(224, 224, 3),
#     include_top=FIXME)

from tensorflow import keras

base_model = keras.applications.VGG16(
    weights='imagenet',
    input_shape=(224, 224, 3),
    include_top=False)



### 冻结基础模型
接下来，我们建议您像在笔记本05b中一样冻结基础模型。这样做是为了使从ImageNet数据集中所学到的知识都不会在初始的训练中被破坏。

In [17]:
# Freeze base model
# base_model.trainable = FIXME

base_model.trainable = False


### 向模型添加新层
现在该向预训练模型中添加新层了。您可以再次使用笔记本05b作为指导。请密切注意最后的全连接（Dense）层，并确保其具有正确数量的神经元以对不同类型的水果进行分类。

In [18]:
# Create inputs with correct shape
# inputs = FIXME
inputs = keras.Input(shape=(224, 224, 3))

x = base_model(inputs, training=False)

# Add pooling layer or flatten layer
# x = FIXME
x = keras.layers.GlobalAveragePooling2D()(x)

# Add final dense layer
# outputs = keras.layers.Dense(FIXME, activation = 'softmax')(x)
outputs = keras.layers.Dense(1, activation = 'softmax')(x)

# Combine inputs and outputs to create model
# model = keras.Model(FIXME)
model = keras.Model(inputs, outputs)


In [19]:
model.summary()


Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
vgg16 (Model)                (None, 7, 7, 512)         14714688  
_________________________________________________________________
global_average_pooling2d_1 ( (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 513       
Total params: 14,715,201
Trainable params: 513
Non-trainable params: 14,714,688
_________________________________________________________________


### 编译模型
现在可以使用损失函数（loss）和衡量标准（metrics）选项来编译模型了。请记住，我们正在训练的模型是要解决多分类而不是二分类的问题。

In [25]:
# model.compile(loss = FIXME , metrics = FIXME)
model.compile(loss='categorical_crossentropy', metrics=['accuracy'])

### 扩充数据
如果需要，请尝试扩充数据以改进数据集。请参考笔记本04a和笔记本05b中的数据扩充的示例。您也可以查看[Keras ImageDataGenerator类](https://keras.io/api/preprocessing/image/#imagedatagenerator-class)的文档。 此步骤是可选的，但是您可能会发现，这对训练时能达到95％的准确率很有帮助。

In [26]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# datagen = ImageDataGenerator(FIXME)

datagen = ImageDataGenerator(
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images horizontally
        vertical_flip=False)  # Don't randomly flip images vertically

### 加载数据集
现在应该加载训练和测试数据集了。您必须选择正确的文件夹以及图像的正确的`target_size`（它必须与您创建的模型的输入高度和宽度相匹配）。如果您需要参考，可以查看笔记本05b。

In [27]:
# # load and iterate training dataset
# train_it = datagen.flow_from_directory(FIXME, 
#                                        target_size=FIXME, 
#                                        color_mode='rgb', 
#                                        class_mode="categorical")
# # load and iterate validation dataset
# valid_it = datagen.flow_from_directory(FIXME, 
#                                       target_size=FIXME, 
#                                       color_mode='rgb', 
#                                       class_mode="categorical")

# load and iterate training dataset
train_it = datagen.flow_from_directory('data/fruits/train/', 
                                       target_size=(224, 224), 
                                       color_mode='rgb', 
                                       class_mode="categorical")
# load and iterate validation dataset
valid_it = datagen.flow_from_directory('data/fruits/valid/', 
                                      target_size=(224, 224), 
                                      color_mode='rgb', 
                                      class_mode="categorical")


Found 1182 images belonging to 6 classes.
Found 329 images belonging to 6 classes.


### 训练模型
现在开始训练模型！将训练和测试数据集传递给`fit`函数，并设置所需的训练次数（epochs）。

In [28]:
# model.fit(FIXME,
#           validation_data=FIXME,
#           steps_per_epoch=train_it.samples/train_it.batch_size,
#           validation_steps=valid_it.samples/valid_it.batch_size,
#           epochs=FIXME)

model.fit(train_it, 
          validation_data=valid_it, 
          steps_per_epoch=train_it.samples/train_it.batch_size,
          validation_steps=valid_it.samples/valid_it.batch_size,
          epochs=20)


Epoch 1/20
37/36 [==============================] - 21s 557ms/step - loss: 0.0535 - accuracy: 0.9935 - val_loss: 0.1354 - val_accuracy: 0.9873
Epoch 2/20
37/36 [==============================] - 20s 542ms/step - loss: 0.0354 - accuracy: 0.9951 - val_loss: 0.1689 - val_accuracy: 0.9848
Epoch 3/20
37/36 [==============================] - 20s 547ms/step - loss: 0.0394 - accuracy: 0.9952 - val_loss: 0.1329 - val_accuracy: 0.9878
Epoch 4/20
37/36 [==============================] - 20s 546ms/step - loss: 0.0246 - accuracy: 0.9975 - val_loss: 0.1670 - val_accuracy: 0.9889
Epoch 5/20
37/36 [==============================] - 20s 551ms/step - loss: 0.0253 - accuracy: 0.9970 - val_loss: 0.1263 - val_accuracy: 0.9899
Epoch 6/20
37/36 [==============================] - 20s 549ms/step - loss: 0.0163 - accuracy: 0.9987 - val_loss: 0.1110 - val_accuracy: 0.9899
Epoch 7/20
37/36 [==============================] - 20s 549ms/step - loss: 0.0155 - accuracy: 0.9989 - val_loss: 0.1065 - val_accuracy: 0.9873

### 解冻模型以进行微调
如果您已经达到了92％的验证准确率，则此步是可选的。如果没有，我们建议您以很小的学习率尝试对模型进行微调。您可以再次使用笔记本05b作为参考。

In [12]:
# Unfreeze the base model
# base_model.trainable = FIXME
base_model.trainable = True

# Compile the model with a low learning rate
# model.compile(optimizer=keras.optimizers.RMSprop(learning_rate = FIXME),
#               loss = FIXME , metrics = FIXME)
model.compile(optimizer=keras.optimizers.RMSprop(learning_rate = .00001),  # Very low learning rate
              loss='categorical_crossentropy', metrics=['accuracy'])


In [13]:
# model.fit(FIXME,
#           validation_data=FIXME,
#           steps_per_epoch=train_it.samples/train_it.batch_size,
#           validation_steps=valid_it.samples/valid_it.batch_size,
#           epochs=FIXME)

model.fit(train_it, 
          validation_data=valid_it, 
          steps_per_epoch=train_it.samples/train_it.batch_size,
          validation_steps=valid_it.samples/valid_it.batch_size,
          epochs=20)

Epoch 1/20
148/147 [==============================] - 30s 204ms/step - loss: -1.2942 - binary_accuracy: 0.1413 - val_loss: -1.2703 - val_binary_accuracy: 0.1672
Epoch 2/20
148/147 [==============================] - 24s 162ms/step - loss: -1.3128 - binary_accuracy: 0.1421 - val_loss: -1.3190 - val_binary_accuracy: 0.1550
Epoch 3/20
148/147 [==============================] - 24s 162ms/step - loss: -1.2942 - binary_accuracy: 0.1438 - val_loss: -1.2308 - val_binary_accuracy: 0.1429
Epoch 4/20
148/147 [==============================] - 24s 162ms/step - loss: -1.2993 - binary_accuracy: 0.1438 - val_loss: -1.3038 - val_binary_accuracy: 0.1398
Epoch 5/20
148/147 [==============================] - 24s 163ms/step - loss: -1.3043 - binary_accuracy: 0.1421 - val_loss: -1.3433 - val_binary_accuracy: 0.1459
Epoch 6/20
148/147 [==============================] - 24s 163ms/step - loss: -1.3103 - binary_accuracy: 0.1430 - val_loss: -1.2734 - val_binary_accuracy: 0.1429
Epoch 7/20
148/147 [==============

### 评估模型

希望您现在拥有的模型具有92％或更高的验证准确率。如果没有，您可能需要返回并对模型进行更多的训练，或者对数据增强进行调整。

对验证精度满意后，您可以通过执行以下单元格来评估模型。`evaluate`函数将返回一个元组（tuple），其中第一个值是您的损失，第二个值是您的准确率。您需要获得0.92或更高的精度值。

In [29]:
model.evaluate(valid_it, steps=valid_it.samples/valid_it.batch_size)


11/10 [================================] - 4s 355ms/step - loss: 0.0467 - accuracy: 0.9939


[0.046691492199897766, 0.9939209818840027]

### 执行评估

请执行以下2个代码单元来评估您的结果。

**注意：** `run_assessment` 假设您的模型是以 `model` 命名的，而且您的测试数据集的名字是`valid_it`。无论出于什么原因您修改了上述名字，请在下面的单元中对`run_assessment`的参数做相应的修改。

In [30]:
from run_assessment import run_assessment

In [31]:
run_assessment(model, valid_it)

Evaluating model 5 times to obtain average accuracy...

11/10 [================================] - 4s 359ms/step - loss: 0.1014 - accuracy: 0.9919

Accuracy required to pass the assessment is 0.92 or greater.
Your average accuracy is 0.9910.

Congratulations! You passed the assessment!
See instructions below to generate a certificate.


### 生成证书

如果您通过了评估，请返回课程页面（见下图）并单击Assess（评估）按钮，就会产生本课程的合格证书。

<img src="./images/assess_task.png" style="width: 800px;">